<a href="https://colab.research.google.com/github/Francesco146/machine-learning-project/blob/master/Machine_Learning_Pasotto_Marastoni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200
!rm -rf tiny-imagenet-200.zip

--2025-01-11 13:59:18--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-01-11 13:59:18--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  6.83MB/s    in 29s     

2025-01-11 13:59:47 (8.23 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]

test  train  val  wnids.txt  words.txt


In [4]:
import os
import pandas as pd


DATA_DIR = 'tiny-imagenet-200'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VAL_DIR = os.path.join(DATA_DIR, 'val')

val_data = pd.read_csv(f'{VAL_DIR}/val_annotations.txt',
                       sep='\t',
                       header=None,
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])

val_data.head()

val_img_dir = os.path.join(VAL_DIR, 'images')

fp = open(os.path.join(VAL_DIR, 'val_annotations.txt'), 'r')
data = fp.readlines()

val_img_dict = {}
for line in data:
  words = line.split('\t')
  val_img_dict[words[0]] = words[1]
fp.close()

{k: val_img_dict[k] for k in list(val_img_dict)[:10]}

for img, folder in val_img_dict.items():
  new_path = (os.path.join(val_img_dir, folder))
  if not os.path.exists(new_path):
    os.makedirs(new_path)
  if os.path.exists(os.path.join(val_img_dir, img)):
    os.rename(os.path.join(val_img_dir, img), os.path.join(new_path, img))
class_to_name_dict = dict()
fp = open(os.path.join(DATA_DIR, 'words.txt'), 'r')
data = fp.readlines()
for line in data:
  words = line.strip('\n').split('\t')
  class_to_name_dict[words[0]] = words[1].split(',')[0]
fp.close()

{k: class_to_name_dict[k] for k in list(class_to_name_dict)[:20]}

{'n00001740': 'entity',
 'n00001930': 'physical entity',
 'n00002137': 'abstraction',
 'n00002452': 'thing',
 'n00002684': 'object',
 'n00003553': 'whole',
 'n00003993': 'congener',
 'n00004258': 'living thing',
 'n00004475': 'organism',
 'n00005787': 'benthos',
 'n00005930': 'dwarf',
 'n00006024': 'heterotroph',
 'n00006150': 'parent',
 'n00006269': 'life',
 'n00006400': 'biont',
 'n00006484': 'cell',
 'n00007347': 'causal agent',
 'n00007846': 'person',
 'n00015388': 'animal',
 'n00017222': 'plant'}

In [7]:
import torch
import torchvision
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder

class TinyImageNet(Dataset):
  _TRAIN_DIR = TRAIN_DIR
  _VAL_DIR = val_img_dir

  def __init__(self, split = 'train', transform = None):
    super().__init__()
    self.transform = transform

    match split:
      case 'train':
        dataset = ImageFolder(root=self._TRAIN_DIR, transform=transforms.ToTensor())
      case 'val':
        dataset = ImageFolder(root=self._VAL_DIR, transform=transforms.ToTensor())
      case _:
        raise ValueError(f'Invalid split: {split}')

    self.imgs_path = np.array(dataset.imgs)
    self.labels = np.array(dataset.targets)
    self.n_classes = len(dataset.classes)

  def __len__(self):
    return len(self.imgs_path)

  def __getitem__(self, idx):
    img_path, label = self.imgs_path[idx]
    img = torchvision.io.read_image(img_path)
    if self.transform:
      img = self.transform(img)
    return img, label
